<a href="https://colab.research.google.com/github/giuliocapecchi/LM-project/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDF Preprocessing and Documents Creations

---

Nowadays one of the challenging aspects of preprocessing scientific documents, for the implementation of a RAG (*Retrieval-Augmented Generation*) system, is the accurate **extraction of mathematical formulas**. Considering working with files containing this type of information, we faced the need to adopt a solution that would allow us to achieve results as good and reliable as possible using free tools and resources.

In addition to this, for completeness we report other main relevant tasks, regarding the processing of files in order to implement a RAG system, we tried to manage:
*   **handle complex structures** mantaning contextual meaning;
*   **text cleaning** removing irrilevant elements;
*   **chunk size balancing** accordingly with the specific context limit for the chosen LLM;
*   **text splitting** avoiding breaking meaningful connections in sentences.

For this purpose we propose an hybrid approach that combines multiple solutions. In particular we built a pipeline that integrates functionalities of two solutions:
*   `Unstructure`, an open-source tool efficient in transforming unstructured data into structured outputs.
    *   https://github.com/Unstructured-IO
*   `Nougat`, an advanced Transformer-based OCR model that simplifies the process of converting complex scientific contents into a common and machine-readable format.
    *   https://github.com/facebookresearch/nougat

The tasks we have addressed to these tools are respectively:
1.   Parse the PDF file obtaning **elements characterized by type and content** through `Unstructure partition function`. This allowed us to:
    *   clean the extracted contents **filering out irrelevant or meaningless types** of elements;
    *   identify page breaks, permetting to implement a **page content-aware solution**;
    *   **locate the formulas**, saving indexes of pages containing them;
    *   translate elements into **a dataframe, with columns Type and Value**, for easier handling.

2.   Extract formulas in more accurate and replicable Latex format thanks to `Nougat`. This was done according to the following logic:
    *   **only pages in** which we know **there is at least one formula are processed**, allowing us to save resources and time by avoiding analyzing the pages of the file in which no formulas appear;
    *   the selected pages are converted into images, analyzed using a **Transformer-based OCR model**, and returned as text formatted in LaTeX;
    *   the model's output is analyzed to identify **Latex patterns matching formulas**, which are then extracted.

This approach was necessary because, despite employing a high-resolution element identification strategy in Unstructured partitioning, the formulas often turned out incorrect and failed to meet our expectations. Therefore, we retained the use of `Unstructured` for its ability to implement an **elements and page-aware solution** while integrating `Nougat` in a specialized manner **for extracting mathematical formulas**. This combination allowed us to achieve significantly improved and optimal results.

Continuing with the numbering used above, the final processing steps are:
3.   **Replace the more precise formulas in the dataframe** where all the elements are stored. By doing this we faced the problem of mismatch between expected and extracted formulas for a single page:
    *   if no formulas extracted by `Nougat` we keep the formulas as extracted by `Unstructure`;
    *   else, if the number of extracted formulas differs from the expected one, we give priority to the formulas extracted by `Nougat`, due to more precision and accuracy, discarding the ones extracted by `Unstructure`.
4.   **Page-based Chunks creation**. This was done according to the following logic:
    *   setting a **threshold as the maximum number of tokens** that can be included into a single chunk;
    *   creating chunks by **concatenating page-by-page content** to preserve contextual integrity as much as possible;
    *   if the content of a single page exceeds the threshold, we concatenate **elements one by one until the content remains under the limit**. This approach is a tradeoff between the balancing of chunk sizes and the coherence of contextual structures.

Let's dive into the implementation.


Installing required libraries.

In [1]:
# %pip install -qqq torch gdown huggingface_hub python-dotenv pymupdf python-Levenshtein nltk chromadb tqdm unidecode gradio bitsandbytes seaborn unstructured[pdf] latex2mathml --progress-bar off

### Installing Additional Required Libraries

On Colab, the installations above are sufficient. However, on **Windows** and **macOS**, additional steps are required to install libraries necessary for **Unstructured** and **Nougat**:

- **Poppler**: a PDF rendering library.  
- **Tesseract**: an open-source OCR engine.

---

#### **Steps for Windows**
The easiest way to install the required libraries on Windows is via Conda (assuming Conda is installed):

1. **Install the required packages**:
    ```bash
    %conda install -c conda-forge poppler tesseract -y
    %pip install -q python-magic-bin==0.4.14
    ```

2. **Add Tesseract to the system PATH**:
    Use the following Python code to set up the `TESSDATA_PREFIX` environment variable:

    ```python
    import os, sys

    # Get the path of the current Conda environment
    conda_env_path = os.path.dirname(sys.executable)
    tessdata_path = os.path.join(conda_env_path, "share", "tessdata") # the necessary files should be here

    # Set the TESSDATA_PREFIX environment variable
    os.environ["TESSDATA_PREFIX"] = os.path.join(conda_env_path, "share")
    print("TESSDATA_PREFIX:", os.environ["TESSDATA_PREFIX"])

    # Verify the tessdata directory
    if os.path.exists(tessdata_path):
        print(f"Tessdata directory found: {tessdata_path}")
    else:
        print(f"Tessdata directory not found: {tessdata_path}")
    ```

#### **Steps for macOS**
On macOS, installation is simpler. Use the following command to install the required libraries:

1. **Install the required packages**:
    ```bash
    brew install poppler tesseract
    ```

2. **Verify the installation**:
    Check if Tesseract is properly installed by running:
    ```bash
    tesseract --version
    ```

---

Still, this approach serves as an enhancement to the RAG pipeline, enabling more accurate extraction from PDFs, particularly for elements like mathematical formulas in scientific papers. However, it is not mandatory; if the PDF contains mostly plain text, a simpler chunking method may be sufficient and more straightforward.

In [ ]:
import nltk

print(nltk.__version__)  # must be 3.9.1

nltk.download("punkt")

Let's download the selected PDF for this analysis. You can choose any PDF you prefer, but keep in mind that your evaluation phase will have to differ.

> **Note:** To speed up execution, we’ll also download the `elements.pkl` file to bypass some computationally expensive processing (which would otherwise take approximately *20/30 minutes* for ~300 pages).

In [ ]:
import os
import gdown


PDF_NAME = "Information Retrieval Slides.pdf"
if os.path.exists(PDF_NAME):
    print(f"File {PDF_NAME} already exists.")
else:
    # save locally from https://drive.google.com/file/d/1xUA6_ZBJzWGF7kWpM1YZTK3R1siYg1qY/view?usp=drive_link
    gdown.download(id="1xUA6_ZBJzWGF7kWpM1YZTK3R1siYg1qY", output=PDF_NAME, quiet=False)

    # download of pickled elements to speed up the execution
    gdown.download(id="17eXYgmiTL9-f9F5vIT5j7QffHIu_QwuN", output="elements.pkl", quiet=False)

## PDF Partitioning with Unstructured
Processes the PDF using the `partition` function to extract structured elements.

We decide to apply the **high-resolution strategy** because is highly sensitive, it is recommended if you want obtain precise classifications for document elements. Adopting this strategy we are able to use the document layout to gain additional information about document elements, in particular we are able to recognize:
- **Titles** and **Text** (we use them as it is)
- **Formulas** (we further process them)
- **Images** (will be removed)
- **Page Breaks** (fundamental to handle formulas and create chuncks)


In [ ]:
from unstructured.partition.auto import partition
from tqdm import tqdm
import fitz  # PyMuPDF
import pickle

file_name = "Information Retrieval Slides.pdf"
title = "Information Retrieval"

load_elements_from_pickle = True
elements = []

if load_elements_from_pickle:
    # Loading the pickled elements.
    with open('elements.pkl', 'rb') as f:
        elements = pickle.load(f)
else: # Process the PDF file
    doc = fitz.open(file_name)  # open the PDF file
    total_pages = len(doc)

    for page_number in tqdm(range(total_pages), desc="Processing PDF pages"):
        # Crea un nuovo documento con una sola pagina
        temp_doc = fitz.open()  # Documento vuoto
        temp_doc.insert_pdf(doc, from_page=page_number, to_page=page_number)

        # Salva la singola pagina in un file temporaneo
        temp_page_file = f"temp_page_{page_number}.pdf"
        temp_doc.save(temp_page_file)
        temp_doc.close()

        # Applica `partition` sulla singola pagina
        page_elements = partition(
            filename=temp_page_file,
            strategy="hi_res",
            skip_infer_table_types=[],
            include_page_breaks=True,
        )
        elements.extend(page_elements)  # Aggiungi gli elementi alla lista totale
        # delete the temporary file
        os.remove(temp_page_file)

    doc.close()  # Chiudi il documento principale

# Stampa il numero totale di elementi
print("Number of recognized elements: " + str(len(elements)) + " in file " + file_name)

Storing the elements in a pickle file.
Snippet of code executed if needed.

In [5]:
# to uncomment, now is commented cause we are downloading the pickled elements
with open('elements.pkl', 'wb') as f:
    pickle.dump(elements, f)

## Filtering and Analyzing Extracted Elements

Here we build a **dataframe**, creating pairs of **Type** and **Value**, using the elements extracted from the PDF file.

Then we print all the different element's Type recognized.



In [ ]:
import pandas as pd

df_elements = pd.DataFrame( [[type(elem), elem.text] for elem in elements], columns=['Type', 'Value'])

# Print all the unique types of elements
print(df_elements['Type'].unique())

Now we clean the dataframe filtering out all the useless elements:
- **Images** (for them we have a text extracted with OCR technique, but not very useful for the language model due to a loss of meaning)
- **Table** (similarly to the images they lost some meaning as they are extracted)
- **Footers** and **Headers**


In [ ]:
from unstructured.documents.elements import Image, Table, Footer, Header, PageBreak, Formula

# print df number of rows
print("Number of actual elements: " + str(len(df_elements)))

# drop all the rows with unuseful Types
df_elements = df_elements[df_elements['Type'] != Image]
df_elements = df_elements[df_elements['Type'] != Table]
df_elements = df_elements[df_elements['Type'] != Footer]
df_elements = df_elements[df_elements['Type'] != Header]
df_elements = df_elements.drop(df_elements[(df_elements['Type'] != PageBreak) & (df_elements['Value'] == '')].index)

# print df number of rows
print("Number of elements after filtering: " + str(len(df_elements)))

# reassign the dataframe index
df_elements = df_elements.reset_index(drop=True)

# print all the different possible Type in the dataframe
print("Remaining Types left in the dataframe:\n",df_elements['Type'].unique())


Check if there are some empty-valued elements, that are not Page Breaks, in the dataframe.

In [ ]:
# print rows of df_elements with bith Type != PageBreak and Value empty
print(len(df_elements[(df_elements['Type'] != PageBreak) & (df_elements['Value'] == '')]))

if len(df_elements[(df_elements['Type'] != PageBreak) & (df_elements['Value'] == '')]) > 0:
  df_elements = df_elements.drop(df_elements[(df_elements['Type'] != PageBreak) & (df_elements['Value'] == '')].index)

## Filtering Pages Containing Formulas in a DataFrame

This code processes a DataFrame (`df_elements`) containing elements from a document (e.g., text, formulas, and page breaks) to identify and remove all rows belonging to pages that contain at least one formula.

- `pages_with_formulas_indexes`: A `set` to store the indices of pages that contain at least one `Formula`.
- `index`: Keeps track of the current page number (starts at 0).
- `page_indices`: A list to map each row in the DataFrame to its corresponding page number.


In the end, we filter out rows whose `PageIndex` matches any value in `pages_with_formulas_indexes`. The `PageIndex` column is dropped after filtering.

In [ ]:
# Initial number of PageBreak elements
print("Number of PageBreak elements: " + str(len(df_elements[df_elements['Type'] == PageBreak])))

# list of page indexes with at least one formula
pages_with_formulas_indexes = set()
index = 0
page_indices = [] # to map each row to its page index

for i in range(len(df_elements)):
    # when a PageBreak is found, increment the page-index
    if df_elements.iloc[i]['Type'] == PageBreak:
        index += 1
    page_indices.append(index)
    # if you find a formula, add the page index to the set
    if df_elements.iloc[i]['Type'] == Formula:
        pages_with_formulas_indexes.add(index+1)

# add the PageIndex column to the DataFrame
df_elements['PageIndex'] = page_indices

# filter out the rows with formulas
df_elements_cleaned = df_elements[~df_elements['PageIndex'].isin(pages_with_formulas_indexes)].drop(columns=['PageIndex']).reset_index(drop=True)

# Final number of PageBreak elements
print("Number of PageBreak elements: " + str(len(df_elements_cleaned[df_elements_cleaned['Type'] == PageBreak])))
print("Pages with formulas indexes:", sorted(pages_with_formulas_indexes))
print("Number of pages with at least one formula:", len(pages_with_formulas_indexes))


In [ ]:
print("Number of elements before filtering:", len(df_elements))
print("Number of elements after filtering:", len(df_elements_cleaned))

The Unstructured pipeline can make some mistakes in extracting text from the pdf. This code replaces uncommon Unicode characters with ASCII equivalents and removes any remaining non-ASCII characters. We defined a dictionary for replacing specific Unicode characters (e.g., `ﬃ → ffi`, `⊺ → T`), more can be added if necessary. We use `str.maketrans()` for fast replacements. The computed values are stored back in the `Value` column of the dataframe.

In [11]:
import re

unicode_replacements = {
    "ﬃ": "ffi",
    "ﬀ": "ff",
    "ﬁ": "fi",
    "⊺": "T",
    "•": " ",
    "—": "-",
    "«": " ",
    "»": " ",
    "”": '"',
    "“": '"',
    "‘": "'",
    "’": "'",
    "ﬄ": "ffl",
    # Add more as needed
}

# Create a translation table for faster replacement
translation_table = str.maketrans(unicode_replacements)

def find_non_ascii_characters(text):
    # Replace all characters using the translation table
    text = text.translate(translation_table)
    # Optional: Remove any remaining non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]', '', text)  # Removes all remaining non-ASCII characters
    return text

# Remove non-ASCII characters from each element's text
for i in range(len(df_elements_cleaned)):
    df_elements_cleaned.loc[i, 'Value'] = find_non_ascii_characters(df_elements_cleaned.loc[i, 'Value'])

## Nougat: Neural Optical Understanding for Academic Documents

[**Nougat**](https://github.com/facebookresearch/nougat?tab=readme-ov-file) is an academic document PDF parser designed to extract and understand LaTeX math and tables from PDFs. Developed by **Facebook Research**, it converts academic documents into structured formats (Mathpix Markdown, *.MMD*).

Key Features:
- Extracts and processes LaTeX math, tables, and text.
- Converts PDFs to Markdown (.mmd) format.
- Supports GPU for faster processing.

Nougat is optimized for scientific papers, especially those found on platforms like arXiv, and works best with English papers.

The idea now is to employ it as a *bullet-proof* approach for extracting text difficult pages, like the ones with mathemathical formulas (which indexes were saved above).

> As a side note, unfortunately Nougat requires `transformers==4.38.2` or below, which is uncompatible with `sentence-transformers`, since it requires `transformers<5.0.0,>=4.41.0`. A solution to this is to keep them separated in two different virutal environments. For the sake of semplicity, in the notebook we will simply upgrade `transformers` later on in order to utilize `sentence-transformers` without any problems.

In [ ]:
# install some pre-requisites for Nougat
%pip install -qqq -U albumentations
%pip -qqq install transformers==4.38.2 --progress-bar off

# install Nougat
%pip -qqq install nougat-ocr  --progress-bar off

Now we can simply invoke Nougat, by calling the corresponding CLI command. More information about it can be found on its [README page](https://github.com/facebookresearch/nougat?tab=readme-ov-file). Here it is called by simply passing:
- the path of the PDF file in analysis
- `-o`, the output folder
- `--pages`, to specify only pages containing mathemathical formulas
- `-m 0.1.0-base`, to utilize the base model (default to `0.1.0-small` but didn't produce acceptable results)
- `--no-skipping`, to avoid using failure detection heuristic
- `--recompute`, to recompute already computed PDF, discarding previous predictions.
> Requires ~1.3 minutes for 22 pages with formulas

In [ ]:
page_with_formula_indexes_str = ",".join(map(str, sorted(pages_with_formulas_indexes)))
print(page_with_formula_indexes_str)

command = f"nougat \"{file_name}\" -o ./nougat-output --pages {page_with_formula_indexes_str} -m 0.1.0-base --no-skipping --recompute"

print(command)
!{command}

The output can be found in the `nougat-output` folder, as an MMD file. Since of course the model can fail to identify some sentences, we also perform some preprocessing of the text, to ensure it will be sanitized for the next phases.

In [ ]:
def remove_unmatched_left_tags(line):
        if "\\left|" in line and "\\right|" not in line:
            line = line.replace("\\left|", "")
        return line

def preprocess_content(content):
    content = content.replace("**", "").replace("##", "")  # Some markdown is left sometimes
    content = content.replace("{(}", "(").replace("{)}", ")") # replace all '{(}' and '{)}' with '(' and ')'
    content = content.replace("{[}", "(").replace("{]}", ")") # replace all '{[}' and '{]}' with '(' and ')'
    content = re.sub(r'\$(.*?)\$', r'\\(\1\\)', content) # replace all '$' with '\(\)' (inline formulas)
    content = re.sub(r'\\text\{(.*?)\}', r'\1', content) # replace all '\text' with ''
    content = re.sub(r'_(.*?)_', r'\1', content) # replace words inside '_word_' with the same word -> e.g. _Proof:_ -> Proof:
    content = re.sub(r'\\includegraphics\[width=.*?pt\]', '', content) # replace all \includegraphics[width=....pt] with '', independently of the width
    content = "\n".join(remove_unmatched_left_tags(line) for line in content.splitlines()) # remove all \left|\ tags that don't have a \right|\ tag in the same line (they are not closed)        
    content = content.strip()
    return content


with open(f'nougat-output/{file_name.replace(".pdf",".mmd")}', 'r') as f:
    content = f.read()

print(content[:200], " ...\n\n..........................................")

processed_content = preprocess_content(content)
 
with open(f'nougat-output/{file_name.replace(".pdf",".mmd")}', 'w') as f:
    f.write(processed_content)


Now the text is ready to be processed with Unstructured as before, which will be precise this time, since it doesn't have to extract text from the PDF but just to take it *as-is*.

In [15]:
from unstructured.partition.text import partition_text
from unstructured.cleaners.core import group_broken_paragraphs
from unstructured.documents.elements import Title, PageBreak

result = partition_text(text=processed_content, paragraph_grouper=group_broken_paragraphs)

To make it homogeneous to the rest of the text produced before, we split the output in "pages" by simply adding a PageBreak before each Title

In [ ]:
updated_result = []

for e in result:
    if isinstance(e, Title): # if the element is a Title ...
        updated_result.append(PageBreak(""))  # ... add a PageBreak before it
    updated_result.append(e)

# Print how many PageBreaks are in the updated_result
print("Number of PageBreaks in the updated_result: ", len([e for e in updated_result if isinstance(e, PageBreak)]),"\n")
print("Number of elements in the updated_result: ", len(updated_result),"\n")

# Print the first 20 elements of the updated_result
for e in updated_result[:20]:
    print(f"{type(e).__name__}: {e.text}")

Finally the produced text with formulas is re-added to the dataframe

In [17]:
df_elements = pd.concat([df_elements_cleaned, pd.DataFrame([[type(elem), elem.text] for elem in updated_result], columns=['Type', 'Value'])], ignore_index=True)

# Creating the chunks

Let's start by cleaning up the `df_elements` DataFrame by removing consecutive duplicate rows (keeping the first occurence). This can happen because... TODO boh ? non dovrebbe in realtà quindi non ho idea del motivo


In [ ]:
print("Number of elements before removing adjacent duplicates: " + str(len(df_elements)))

# drop adjacent duplicates in the df_elements mantaining the first one
df_elements = df_elements.loc[df_elements.shift().ne(df_elements).any(axis=1)]

# reset index
df_elements = df_elements.reset_index(drop=True)

print("Number of elements after removing adjacent duplicates: " + str(len(df_elements)))

In [ ]:
print("Final number of pages (given by the number of PageBreak(s)):",len(df_elements[df_elements['Type'] == PageBreak]))

In [ ]:
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')

def count_tokens(text):
  return len(word_tokenize(text))

Here we compute some statistics regarding the number of tokens in the pages.

In [ ]:
# compute the max, the min and the average number of tokens in a page by concatenating elements of the dataframe between consecutive page breaks

num_token_per_page = []

current_page = ""
current_page_tokens = 0

for i in range(len(df_elements)):
  if df_elements.iloc[i]['Type'] != PageBreak:
    current_page += df_elements.iloc[i]['Value']
    current_page_tokens = count_tokens(current_page)
  else:
    num_token_per_page.append(current_page_tokens)
    current_page = ""
    current_page_tokens = 0

print("Max tokens: " + str(max(num_token_per_page)))
print("Min tokens: " + str(min(num_token_per_page)))
print("Avg tokens: " + str(sum(num_token_per_page)/len(num_token_per_page)))

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

mean_tokens = np.mean(num_token_per_page)

plt.figure(figsize=(9, 5))
plt.hist(num_token_per_page, bins=10, edgecolor='black', alpha=0.5)
plt.axvline(mean_tokens, color='red', linestyle='dashed', linewidth=1, label=f'Mean = {mean_tokens:.2f}')
plt.title('Distribution of Number of Tokens per Page')
plt.xlabel('Number of Tokens')
plt.ylabel('Frequency')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Splitting Content into Tokenized Chunks
Here we divide the processed elements in `df_elements` into smaller text files, ensuring each file stays within a token limit.

**Chunk Creation:**
- Processes elements in `df_elements`:
  - Concatenates values until the token count exceeds `doc_tokens_threshold`.
  - Writes each chunk to a separate text file (`docX.txt`).
  - Tracks the token count of each chunk in `docs_len`.
- Handles `PageBreak` to reset and manage document boundaries.


>NOTE: `doc_tokens_threshold` can be adjusted to change the maximum token count per file.


In [ ]:
import os

# create folder chunks if it doesn't exist
if not os.path.exists('chunks'):
    os.makedirs('chunks')
else:
    # delete all the files in the folder chunks
    for file in os.listdir('chunks'):
        os.remove(os.path.join('chunks', file))


block = ""
document = ""
doc_index = 0
doc_tokens_threshold = 150
docs_len = []

# iterating over df_elements
for i in range(len(df_elements)):


  if df_elements.at[i, 'Type'] != PageBreak:

    next_element = df_elements.at[i, 'Value'] + " "

    if count_tokens(block) + count_tokens(next_element) > doc_tokens_threshold:
        with open('chunks/document_' + str(doc_index) + '.txt', 'w') as f:
          f.write(block)

        docs_len.append(count_tokens(block))
        doc_index += 1
        block = ""

    block += next_element

  else: # if the type is a PageBreak

    expected_num_tokens = count_tokens(document) + count_tokens(block)
    if expected_num_tokens > doc_tokens_threshold:

      with open('chunks/document_' + str(doc_index) + '.txt', 'w') as f:
        f.write(document)

      docs_len.append(count_tokens(document))
      doc_index += 1
      document = ""

    document += block
    block = ""

print("Number of chunks obtained: " + str(doc_index))

In [ ]:
# print the max, the min and the avg of values in docs_len
print("Number of documents: ",len(docs_len))
print("Maximum number of tokens in a document: ",max(docs_len))
print("Minimum number of tokens in a document: ",min(docs_len))
print("Average number of tokens in a document: ",sum(docs_len)/len(docs_len))

In [ ]:
# plot the distribution of number of tokens per document showing the mean
mean_tokens = np.mean(docs_len)

plt.figure(figsize=(9, 5))
plt.hist(docs_len, bins=10, edgecolor='black', alpha=0.5)
plt.axvline(mean_tokens, color='red', linestyle='dashed', linewidth=1, label=f'Mean = {mean_tokens:.2f}')
plt.title('Distribution of Number of Tokens per Document')
plt.xlabel('Number of Tokens')
plt.ylabel('Frequency')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()

# RAG Pipeline on PDFs with Limited GPU Requirements

In this project, we aim to build a **Retrieval-Augmented Generation (RAG)** pipeline optimized for **limited GPU environments**. The goal is to combine the power of *dense retrieval-based methods* with the flexibility of generative models, while ensuring the system remains efficient enough to run on medium-tier laptops with GPUs.

A RAG pipeline consists of two main stages: **retrieval** and **generation**. The entire process can be divided into **3 key steps**:

1. **Generation of document embeddings**  
   In this step, document embeddings are generated from a *corpus* using an embedding model. Each document is encoded into a dense vector representation, capturing semantic information.

2. **Document retrieval**  
   Relevant documents are fetched from the corpus based on the input query. This is achieved by utilizing the generated embeddings and calculating similarity with the query's embedding to retrieve the most relevant documents.

3. **Output generation**  
   The retrieved documents are then passed to a **generative model**, which produces contextually relevant responses based on the information extracted from the documents.

By leveraging lightweight models and optimizing for efficiency, this RAG pipeline delivers powerful **AI-driven results** even on hardware with **limited resources**.

Finally, a **Gradio interface** is provided, allowing users to interact with the system and "chat" with the documents they provide, offering a seamless experience for exploring the content.

## Hardware requirements and constraints

Let's find out what hardware we've got available to see what kind of model(s) we'll be able to load. You can also check this with the `!nvidia-smi` command.

> **NOTE** : as said above, `sentence-transformers` requires a different version of the `transformers` library, so we update it here.

In [ ]:
%pip install -qqq -U transformers

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Get GPU available memory
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")
print(torch.cuda.get_device_name(0))

Of course, depending on the provided harware, better models can be utilized.

### Checking local GPU memory availability

Let's first analyze how we decided the model to use for this project. This notebook was primarily run and tested locally on a **laptop** with 16GB of RAM and an NVIDIA RTX 3070 laptop GPU (8GB of VRAM). The main goal was to create a pipeline that could efficiently run on this portable device, leveraging the benefits of GPUs and CUDA for AI tasks while ensuring that the performance and capabilities did not feel lacking compared to larger models.

We will need two main ingredients:
- An *embedder* model, that calculates dense embeddings from documents
- An *LLM*, that provides output given user's queries and the retrieved documents

In their dedicated sections we will uncover the choiches made for both.

### Load our PDF and start producing chunks

Let's now start by loading a pdf file and extracting chunks from it. These chunks' **quality** is important, since these will be essentially the *documents* on which we will compute the embeddings on.

In [ ]:
from huggingface_hub import login
from dotenv import load_dotenv

PDF_NAME = "Information Retrieval Slides.pdf"
running_on_colab = False

try:
    from google.colab import userdata # type: ignore
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("Running in Google Colab. Using userdata to get HF_TOKEN.")
    running_on_colab = True
except ModuleNotFoundError:
    load_dotenv()
    HF_TOKEN = os.getenv('HF_TOKEN')
    print("Not running in Google Colab. Using load_dotenv to get HF_TOKEN.")

file_path = PDF_NAME
login(token=HF_TOKEN)

In [ ]:
# TODO : i chunk sono già in memory, si potrebbero semplicemente usare

corpus = []
# for each document in chunks folder append its text to corpus
for file in sorted(os.listdir("chunks")):
    with open(os.path.join("chunks", file), "r", encoding="utf-8") as f:
        corpus.append(f.read())

print(len(corpus))

Let's give a look to some random documents extracted and preprocessed from the initial pdf

In [ ]:
import random

# print some random pages with their indices
random.seed(42)
for i in range(5):
    doc = random.choice(corpus)
    doc_index = corpus.index(doc)
    print(f"DocId: {doc_index}, {doc[:100]}...")

In [ ]:
print(f"Average document length: {sum(len(doc) for doc in corpus) / len(corpus)}")

We also provide a `save_documents` function, which can be useful for further inspection of the PDF content.

In [33]:
# import os

# def save_documents(documents):
#     """
#     Save the provided documents to the vector_store directory. Each document is saved as a separate text file.
#     """
#     if not os.path.exists("vector_store"):
#         os.makedirs("vector_store")
#     # delete all files in the vector_store directory
#     for file in os.listdir("vector_store"):
#         os.remove(os.path.join("vector_store", file))
#     for i, doc in enumerate(documents):
#         with open(f"vector_store/document_{i}.txt", "w", encoding="utf-8") as f:
#             f.write(doc.page_content)

In [ ]:
#save_documents(corpus)
print(f"Saved {len(corpus)} documents in the vector_store directory.")

Later, we will use two `json` files containing questions to test the pipeline: one with open-ended questions and another following a quiz-like format (multiple-choice questions with four options, where only one is correct). For now, we’ll download these files to prepare for testing and use one of them to ensure the pipeline works as expected.

We’ll begin by defining the function `extract_questions(file_path, question_type)`, which takes two parameters:

- `file_path`: the path to the JSON file containing the questions.
- `question_type`: specifies the type of questions to extract and can be:
  - `'open'` for open-ended questions.
  - `'mc'` for multiple-choice questions.

At this stage, since our primary goal is to verify that everything functions as intended, we will focus on using the multiple-choice JSON file. 

In [ ]:
import json


def extract_questions(file_path, question_type="mc"):
    """
    Extracts questions from a JSON file.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)

    if question_type == "mc":
        return [
            {
                "question": item.get("question", "").strip(),
                "answers": item.get("answers", ""),
                "correct": item.get("correct", ""),
                "question_id": item.get("question_id", "")
            }
            for item in data
        ]
    elif question_type == "open":
        return [
            {
                "question_id": item.get("question_id", ""),
                "question": item.get("question", "")
            }
            for item in data
        ]
    else:
        raise ValueError(f"Unsupported question type: {question_type}")



if not os.path.exists("evaluation"):
    os.makedirs("evaluation")
    # save locally from https://drive.google.com/file/d/1m2_iG7cGOgRwfVaXzTORU7Pcn71UFKcG/view?usp=drive_link
    gdown.download(id="1m2_iG7cGOgRwfVaXzTORU7Pcn71UFKcG", output="evaluation/quiz.json", quiet=False)
    # save locally from https://drive.google.com/file/d/1L6rvrbPtwGduaN-8DmpLAWXUtHSfhhF2/view?usp=sharing
    gdown.download(id="1L6rvrbPtwGduaN-8DmpLAWXUtHSfhhF2", output="evaluation/open_questions.json", quiet=False)

queries = extract_questions("evaluation/quiz.json", "mc")
print(f"Loaded {len(queries)} questions.")

random.seed(4242)
print(random.choice(queries))

## Gather Dataset Embeddings

In this first stage, the aim is to produce the documents embeddings, in order to be able to make similarity searches in the upcoming steps of the pipeline.

### About the `cde-small-v1` Model

The `cde-small-v1` model, developed by John X. Morris and Alexander M. Rush, is a cutting-edge model for generating **Contextual Document Embeddings (CDE)** ([link](https://huggingface.co/jxm/cde-small-v1)). What sets this model apart is its ability to integrate "*context tokens*" into the embedding process, which allows it to capture the nuances and relationships between documents more effectively. This makes it particularly suitable for generating highly accurate embeddings for both documents and queries, especially in cases where capturing the context of a document within the broader corpus is crucial.

We chose this model because, as of December 2024, it is one of the leading models under 400M parameters, delivering impressive results on the [**Massive Text Embedding Benchmark (MTEB) leaderboard**](https://huggingface.co/spaces/mteb/leaderboard). Although it ranks 32nd overall, it stands out as the top model in terms of **memory efficiency**, which is a key factor for our project, given the requirement for **limited GPU capabilities**. Additionally, it offers a substantial **embedding dimension of 768**, striking a balance between computational efficiency and embedding quality.

<style>
    img {
        border-radius: 15px;
    }
</style>
![assets/cde-small-v1.png](https://github.com/giuliocapecchi/LM-project/blob/main/assets/cde-small-v1.png?raw=1)

A key feature of the `cde-small-v1` model is its optimization for a two-stage embedding process:

1. **First Stage**: Embedding a subset of documents from the corpus to create "dataset embeddings," which serve as a **reference** for the entire corpus.
2. **Second Stage**: Using the dataset embeddings to embed new queries and documents during inference.

This model is compact yet delivers solid performance, making it suitable for our use case.

### Steps to Gather Dataset Embeddings:

1. **Selecting a Subset of Documents**  
   We begin by sampling a representative set of documents from the corpus. Following the model's guidelines, we select 512 documents. If this number isn't available, the model can handle oversampling, which is the case for the PDF dataset we're using. Despite this, performance remains strong.

2. **Generating Dataset Embeddings**  
   After selecting the documents, we encode them using the `cde-small-v1` embedding model. This step produces dense vector representations of the documents, which are representative of the broader corpus.

3. **Embedding Queries and Documents**  
   Once the dataset embeddings are created, we use them to embed both documents and queries. A key feature of this model is its ability to differentiate between 'queries' and 'documents' during encoding, ensuring context is preserved during the embedding process.


Next, we will load the model using the `SentenceTransformers` interface to begin the embedding process.

In [ ]:
from sentence_transformers import SentenceTransformer

# with sentence-transformers we don't need prefixes but, to do retrieval, we need to use prompt_name="query" and prompt_name="document" in the encode method of the model when embedding queries and documents, respectively.
embeddings_model = SentenceTransformer(
    "jxm/cde-small-v1",
    trust_remote_code=True,
).to(device)

During the development of the project, the creators of the embedding model made a change in the Hugging Face repository, which led to unstable results when using the `SentenceTransformer` implementation. If a similar issue arises in the future, it can be mitigated by specifying the `revision` and `tokenizer_kwargs` parameters in the model constructor. These parameters allow to lock the model and tokenizer to a specific branch name, tag, or commit ID from the Hugging Face repository, ensuring stability.

Here’s how to implement it:

```python
embeddings_model = SentenceTransformer(
    "jxm/cde-small-v1",
    trust_remote_code=True,
    revision="9e2ed1d8d569d34458913d2d246935c1b2324d11",  # Latest stable model revision
    tokenizer_kwargs={"revision": "86b5e0934494bd15c9632b12f734a8a67f723594"}  # Latest stable tokenizer revision
).to(device)
```

The tags provided above correspond to the latest stable commits (as of December 2024). You can retrieve them directly from the model's card page on Hugging Face, under the "Files and Versions" tab.

Fortunately for us, the issue was resolved the same day we reported it to the creators via Twitter. We thank them for their prompt response and the fix they provided.

Let's follow up by producing the `minicorpus` (which is the subsample of the whole corpus) and what we called `dataset_embeddings`

In [ ]:
minicorpus_size = embeddings_model[0].config.transductive_corpus_size # 512
random.seed(4242)
minicorpus_docs = random.choices(corpus, k=minicorpus_size) # oversampling is okay
assert len(minicorpus_docs) == minicorpus_size # We must use exactly this many documents in the minicorpus

dataset_embeddings = embeddings_model.encode(
    #[doc.page_content for doc in minicorpus_docs],
    [doc for doc in minicorpus_docs],
    prompt_name="document",
    convert_to_tensor=True,
    show_progress_bar=True
)

print(f"Corpus size: {len(corpus)}")
print(f"Computed embeddings for {len(minicorpus_docs)} documents. Shape: {dataset_embeddings.shape}")

In [ ]:
print("Some mini-corpus documents:")

# get some random documents from the minicorpus
random.seed(4242)
for i in random.sample(range(minicorpus_size), 5):
    #print(f"document {i}: {minicorpus_docs[i].page_content[:100]}...")
    print(f"document {i}: {minicorpus_docs[i][:100]}...")

Now that we have obtained the **dataset embeddings**, we can proceed to embed both documents and queries using the same model.

To embed the documents and queries, we must ensure that we specify the correct `prompt_name` for each, as well as pass the `dataset_embeddings` to maintain context. The reason is that, as many state-of-the-art-models, this one was trained with task-specific prefixes:

- For documents, use:  
  `prompt_name="document"`

- For queries, use:  
  `prompt_name="query"`

We also have to always additionally, specify  the `dataset_embeddings`, in order to use the once we produced before.

By doing so, we ensure that the embeddings are generated with the correct context for both retrieval and generation tasks, leveraging the efficient performance of the `cde-small-v1` model.

In [ ]:
doc_embeddings = embeddings_model.encode(
    [doc for doc in corpus],
    prompt_name="document",
    dataset_embeddings=dataset_embeddings, # this is the contexualized embeddings of the minicorpus
    convert_to_tensor=True,
    show_progress_bar=True
)

query_embeddings = embeddings_model.encode(
    [query['question'] for query in queries],
    prompt_name="query",
    dataset_embeddings=dataset_embeddings,  # this is the contexualized embeddings of the minicorpus
    convert_to_tensor=True,
    show_progress_bar=True
)

print(f"Document embeddings shape: {doc_embeddings.shape}")
print(f"Query embeddings shape: {query_embeddings.shape}")

We can now computes similarities between the embeddings and all the queries by simply calling `embeddings_model.similarity` (which uses by default cosine similarity), and inspect some of the results obtained.

In [ ]:
similarities = embeddings_model.similarity(query_embeddings, doc_embeddings)
print("'similarities' shape : ",similarities.shape)
topk_values, topk_indices = similarities.topk(5)

random.seed(4242)
random_queries = random.sample(queries, 2)
for query in random_queries:
    query_idx = queries.index(query)
    print(f"Query: {query['question']}")
    for j, idx in enumerate(topk_indices[query_idx]):
        doc = corpus[idx]
        print(f"Rank {j+1} (Score: {topk_values[query_idx][j]:.4f}, Doc ID: {idx}): {doc[:200]}...")
    print()

## Chroma

While not strictly necessary for this project - since the provided PDF is relatively small - we decided to integrate a vector database to enhance the storage and retrieval of the embeddings produced. This integration becomes particularly valuable when larger files are processed, as they often require splitting the corpus into smaller chunks, with each chunk generating its own embedding. The resulting large number of embeddings makes efficient storage and retrieval mechanisms increasingly important.

After evaluating various options, we selected [**Chroma**](https://www.trychroma.com/) for its simplicity of local deployment and good performances. Chroma is particularly well-suited for managing **large document collections**, providing **faster retrieval** and ensuring that the pipeline remains robust and adaptable to big files.

### Implementation Steps

We begin by creating a Chroma client and verifying if a collection named `'chroma-collection'` already exists locally:  if it does, we delete it to ensure a clean state before initializing a new one.

For reference, a Chroma client can be instantiated in two ways :

- in an **ephemeral way** (in-memory), particularly useful for experimentation, such as testing different embedding functions and retrieval techniques. If data persistence is not a requirement, this approach provides a lightweight and efficient way to get started with Chroma while maintaining flexibility for quick iterations. This can be achieved with the following code:

    ```python
    import chromadb
    client = chromadb.Client()
    ```

- with a **persistent client**. This can be done by providing a  path where Chroma will store its database files on disk, and load them on start.

    ```python
    import chromadb
    client = chromadb.PersistentClient(path="/path/to/save/to")
    ```
    If a `path` is not provided, the default is `.chroma`

More about Chroma can be found in its [documentation](https://docs.trychroma.com/docs/overview/introduction). We choose to utilize the **persistent** approach, so let's follow by instantiating a `PersistentClient`;  if the collection already exists, we delete it to ensure a clean state before initializing a new one.


In [ ]:
from chromadb import PersistentClient, Collection

COLLECTION_NAME = "chroma-collection"

# Initialize the Chroma client
client = PersistentClient(path="./"+COLLECTION_NAME)

# Check if the collection exists and delete it if it does
if COLLECTION_NAME in [col.name for col in client.list_collections()]:
    client.delete_collection(name=COLLECTION_NAME)
    print(f"Collection {COLLECTION_NAME} exists, deleting it")
else:
    print(f"Collection {COLLECTION_NAME} does not exist")

### Integration of `cde-small-v1` with Chroma

Integrating the **`cde-small-v1`** embedding model with Chroma required the creation of a custom `EmbeddingFunction`. This was necessary because **`cde-small-v1`** has a distinct workflow that deviates from standard embedding models. Specifically, as said it uses:

- **`prompt_name`**: This parameter distinguishes between "document" and "query" embeddings, to have proper alignment during retrieval tasks.
- **`dataset_embeddings`**: These provide a reference for conditional embedding generation, improving contextual accuracy in embeddings.

By default, Chroma does not natively support these additional parameters. However, it offers the flexibility of extending the `EmbeddingFunction` protocol, allowing us to integrate the specific requirements of the **`cde-small-v1`** model into Chroma’s workflow.

We can check the type of the elements of the corpus produced by our chunking functions and use it to differentiate between queries and documents.

In [ ]:
print(type(corpus[0]))

In [43]:
from chromadb import Documents, EmbeddingFunction, Embeddings


class CustomEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        """
        Automatically detect if the input is a document or a query and set the mode accordingly.
        :param input: Documents or queries to embed.
        :return: Embeddings.
        """
        # TODO : Detect the mode: "document" if the input is a list of documents, else "query".
        mode = "query" #"document" if isinstance(input[0], langchain_core.documents.base.Document) else "query"

        # If the input is a list of Documents, extract the text.
        #if isinstance(input, list):
        #    input = [doc if isinstance(doc, str) else doc.page_content for doc in input]

        # Generate embeddings using the model with the determined mode
        embeddings = embeddings_model.encode(
            input,
            prompt_name=mode,
            dataset_embeddings=dataset_embeddings,
            convert_to_tensor=True,
        ).cpu().numpy()

        return embeddings.tolist()

# Create an instance of the embeddings function for queries (we already calculated all the embeddings of the documents)
custom_embedding_function = CustomEmbeddingFunction()

Furthermore, while Chroma defaults to using Euclidean distance (`l2`) for nearest-neighbor search, the **`cde-small-v1`** embeddings from `SentenceTransformers` are based on cosine similarity by default. To utilize it, We configure the metadata parameter `"hnsw:space": "cosine"` when creating the collection. This ensures that Chroma uses cosine similarity, aligning with the previous behavior.

<style>
    img {
        border-radius: 15px;
    }
</style>
![chroma-similarity-measures.png](https://github.com/giuliocapecchi/LM-project/blob/main/assets/chroma-similarity-measures.png?raw=1)

Once the collection is configured, we add all the documents and their precomputed embeddings to it.

> **NOTE:** We manually computed the embeddings beforehand (for analysis purposes) in the previous steps. If we had only provided the documents, Chroma would have automatically generated the embeddings using the `CustomEmbeddingFunction`, without the need for precomputation.

In [44]:
# Create a Chroma collection with the custom embedding function
collection: Collection = client.create_collection(
    name=COLLECTION_NAME,
    embedding_function=custom_embedding_function,
    get_or_create=True,
    metadata={"hnsw:space": "cosine"} # l2 is the default but cosine is more suitable for this embedding function
)

# Add the documents and their (already computed) embeddings to the collection
collection.add(
    ids=[str(i) for i in range(len(corpus))],  # Unique identifiers for the documents
    embeddings=doc_embeddings.cpu().numpy(),
    #documents=[doc.page_content for doc in corpus],
    documents=[doc for doc in corpus],
)

In [ ]:
#print(collection.peek()) # returns a list of the first 10 items in the collection
print("Number of documents in the corpus: ",len(corpus))
print("Number of documents in the collection: ",collection.count()) # returns the number of items in the collection

We can finally test `collection.query` to make some calls to the Chroma collection, and we expect similar (ideally, the same) results as above, when we used `model.similarity`.

In [ ]:
query_text = "How many bytes can UTF-8 use to encode a character?"

# create a Document object with the query text
results = collection.query(
    query_texts=query_text,
    n_results=5
)
print(results)

# Print the results with the document IDs and scores
for i, (doc_id, score, doc) in enumerate(zip(results['ids'][0], results['distances'][0], results['documents'][0])):
    print(f"Rank {i+1}: Document ID: {doc_id}, Score: {1-score:.4f}, {doc[:200]}")

... which is exactly identical to the previous ones.

One final thing to note is that in Chroma cosine similarity is calculated as `1 - cosim` ; this is done to ensure that lower values indicate a better position in the ranked list. To obtain results consistent with the previous ones, we subtractde the score from one in the above code.


### Compute scores between queries and documents

Now we can analyze a similarity heatmap between the embeddings of our documents and the provided queries. The results show consistency with the following observations:

- **Few documents match the queries**: This suggests that the matching documents are likely the most relevant ones.
- **Matches often occur in subsequent documents**: This is expected since the corpus was split into chunks based on the document's structure. For example, chapters are separated, so answers to specific questions are more likely to be located within the same chapter or adjacent sections of the document.

Additionally, we provide a distribution of the similarity scores. This helps us assess whether an empirical threshold can be established to determine which documents are truly similar to the provided queries.

In [ ]:
from seaborn import heatmap

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot heatmap
heatmap(similarities.cpu().numpy().T, cmap="jet", ax=axes[0])
axes[0].set_title("Similarity Heatmap")

# Plot histogram
axes[1].hist(similarities.cpu().flatten(), bins=50, color='blue', alpha=0.6)
axes[1].set_title("Distribution of Similarity Scores")
axes[1].set_xlabel("Similarity")
axes[1].set_ylabel("Frequency")
axes[1].axvline(x=0.5, color='red', alpha=0.6, linestyle='--')

plt.tight_layout()
plt.show()

## Loading the LLM

After conducting some research, we chose to work with one of Meta's latest smaller open-source Llama models available at the time of writing this notebook: [Llama-3.2-3B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct), released the *25th of September 2024*. We specifically selected the *instruct* variant because it is pre-trained to follow basic instructions, offering a more user-friendly and fine-tuned experience compared to the standard non-instruct version (which is also available). Below, we review this model's [specifications](https://llamaimodel.com/requirements-3-2/):

| **Llama 3.2 3B Instruct Model Specifications**                    | **Requirement**       | **Details**                                                                                  |
|----------------------------------|-----------------------|----------------------------------------------------------------------------------------------|
| Parameters                       | 3 billion             |                                                                                              |
| Context Length                   | 128,000 tokens        |                                                                                              |
| **Hardware Requirements**        |                       |                                                                                              |
| CPU and RAM                      |                       | CPU: Multicore processor <br> RAM: Minimum of 16 GB recommended                              |
| GPU                              |                       | NVIDIA RTX series (for optimal performance), at least 8 GB VRAM                              |
| **Estimated GPU Memory Requirements** |                       |                                                                                              |
| Higher Precision Modes           | BF16/FP16             | ~6.5 GB                                                                                      |
| Lower Precision Modes            | FP8                   | ~3.2 GB                                                                                      |
|                                  | INT4                  | ~1.75 GB                                                                                     |
| **Software Requirements**        |                       |                                                                                              |
| Software Dependencies            |                       | Frameworks: PyTorch <br> Libraries: Hugging Face Transformers (version 4.45.0 or higher), CUDA |

In [ ]:
print("Is bf16 supported: ",torch.cuda.is_bf16_supported())

The `bitsandbytes` library is a lightweight Python wrapper around CUDA custom functions, in particular 8-bit optimizers, matrix multiplication (LLM.int8()), and 8 & 4-bit quantization functions.

The library includes quantization primitives for 8-bit & 4-bit operations, through `bitsandbytes.nn.Linear8bitLt` and `bitsandbytes.nn.Linear4bit` and 8-bit optimizers through bitsandbytes.optim module.

#### Quantization Choices

We chose a *middle ground* by loading the model with **4-bit quantization** while maintaining **BFloat16** precision for computation.

The use of 4-bit quantization reduces the precision of the model’s weights to just 4 bits per value, significantly lowering memory usage and accelerating inference. This method retains only the most essential information, sacrificing some numerical precision, but allows for larger models to be handled on GPUs with limited memory. Despite the weights being quantized to 4 bits, the model still performs computations in **16-bit floating point (BFloat16)** precision. BFloat16 is a 16-bit format that preserves much of the dynamic range of floating-point operations while requiring less memory than the traditional 32-bit format. This way, while the 4-bit quantization reduces the memory footprint of the model weights, the computation is performed in BFloat16, optimizing performance on modern GPUs that are tailored for BFloat16 operations. This configuration strikes a balance between computational efficiency and numerical precision, enabling fast inferences with minimal memory usage without significant loss in result quality.

Given the hardware constraints, we opted for **4-bit quantization** using `BitsAndBytes` (as explained later). This approach greatly reduces the memory footprint and speeds up inference, ensuring acceptable performance for our use case. Without this configuration, the GPU’s memory usage was consistently at 100%, and inference times were approximately 2-3 minutes per query. With 4-bit quantization, memory usage drops to about 6GB, preventing GPU overload and reducing inference time to around 30 seconds per query, delivering satisfactory results. Further details on the quantization process will be provided later.

However, we also need to consider that both the embedding model and the LLM must be loaded into memory, which adds another layer of complexity to the memory management. This requires careful balancing, as the total memory usage must accommodate both the LLM and the embedding model simultaneously. We will address how we manage this in the next section.

In [ ]:
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer
)

MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"

# Quantization is a technique that reduces the precision of the model’s weights to make it run faster and consume less memory, often at the cost of a slight reduction in model accuracy or quality
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Lower precision reduces memory usage and can speed up inference (maybe try 8)
    bnb_4bit_use_double_quant=True,         # Using double quantization can help reduce the loss in accuracy associated with quantization
    bnb_4bit_quant_type="nf4",              # Normal Float 4-bit quantization, a scheme that may preserve model quality better than straightforward quantization methods
    bnb_4bit_compute_dtype=torch.bfloat16,  # The internal compute dtype used during inference. bfloat16 (BF16) is often chosen because it’s efficient on modern accelerators
    llm_int8_enable_fp32_cpu_offload=True   # Enable FP32 CPU offload
)


llm_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map={"": device},  # ensure all modules are on GPU
    quantization_config=bnb_config,
).to(device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

generation_config = llm_model.generation_config
generation_config.max_new_tokens = 200                    # the maximum number of new tokens the model will generate (long outputs might be more prone to off-topic or repetitive content)
generation_config.min_new_tokens = 1                      # the minimum number of new tokens the model will generate
generation_config.temperature = 0.1                       # it controls the randomness of the generation, lower temp means more deterministic, conservative (less creative) and repetitive answers [about 0.1-1.2]
generation_config.top_p = 0.5                             # nucleus sampling controls how the model picks words based on their cumulative probability, lower value (0.5) means safer, more coherent text but less diverse [about 0.5-0.9]
generation_config.num_return_sequences = 1                # how many separate output sequences are returned for each generation prompt, get multiple different answers in one go, useful for picking the best response from several tries
generation_config.pad_token_id = tokenizer.eos_token_id   # special token IDs that represent padding and the end-of-sequence token. Generally, these are set to ensure the model knows when to stop and how to handle inputs of different lengths
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.repetition_penalty = 2.0                # discourages the model from repeating the same phrases or tokens over and over [about 1.0-2.0] (high value cause the model to avoid some tokens even if they are contextually appropriate)

We loaded various things on the GPU up to know. Still, with the proposed configuration for all the pipeline components, GPU usage should be acceptable.

In [ ]:
# Check GPU usage
gpu_memory_gb = torch.cuda.get_device_properties(device).total_memory / (1024 ** 3)  # in GB
memory_allocated_gb = torch.cuda.memory_allocated(device) / (1024 ** 3)  # in GB
print(f"Memory Allocated: {memory_allocated_gb:.2f} GB")
print(f"Total memory usage: {(memory_allocated_gb / gpu_memory_gb) * 100:.2f}%")

In [ ]:
def retrieve_relevant_documents(query, k=5):
    results = collection.query( # query the Chroma collection
        query_texts=query,
        n_results=k
    )
    return results['documents'][0], results['distances'][0]

# test the function
query = "How many bytes can UTF-8 use to encode a character?"
print(f"Query: {query}")
documents, distances = retrieve_relevant_documents(query)
for doc, distance in (zip(documents, distances)):
    print(f"(score: {1-distance:.4f}) {doc[:200]}")

In [52]:
base_prompt =  '''You are an AI assistant for RAG.
Your task is to provide detailed and accurate answers to user questions based on the provided contexts.
If and only if ALL of the provided context lacks sufficient information, just state: "The provided context does not have the answer." without speculating or providing unsupported answers.
Keep the answers concise and professional and do not add informations that are not asked.

User question: {user_query}

Contexts:
{chunks_information}

Answer:
'''

## Perform Queries to the RAG Pipeline

Now, we're ready to test the pipeline. The following function executes a query by performing the steps outlined below:

1. **Retrieve Relevant Documents**: The function invokes `retrieve_relevant_documents` (defined in the coresponding section) to fetch the top 5 relevant documents to the query, along with their similarity scores
2. **Filter Documents**: Documents with a similarity score below 0.5 (calculated as `1 - score`) are discarded, retaining only those with sufficient relevance. If no documents meet the relevance threshold, a message indicating that no relevant documents were found is returned.
3. **Print the results**: If `print_retrieved_documents` is set to `True`, retrieved relevant documetns are also printed on the terminal.
4. **Prepare the Prompt**: The filtered documents are used to create a prompt, which is passed to the model to generate a response. The prompt includes the user's query and the relevant document chunks.
5. **Model Inference**: The prompt is tokenized and passed to the RAG model. The model generates a response using beam search (with 5 beams) and applies early stopping.
6. **Generate and Return Response**: The function decodes the generated tokens and returns the resulting sequence as the response.

This process ensures that the generated response is grounded in relevant documents, providing more accurate, context-aware answers.

In [104]:
def query_rag_model(query, base_prompt=base_prompt, print_retrieved_documents=True):
    """
    Passes the user query to the RAG model and returns the generated answer.
    """
    documents, scores = retrieve_relevant_documents(query, k=5)

    # Filters out documents with a similarity score below 0.5
    filtered_documents = [doc for doc, score in zip(documents, scores) if 1 - score >= 0.5]
    filtered_scores = [score for score in scores if 1 - score >= 0.5]

    # If no relevant documents are found, return a message
    if not filtered_documents:
        print(f"No relevant documents found for query: {query}")
        return "No relevant documents found."

    if print_retrieved_documents:
        print("Filtered documents and their scores: ")
        for doc, score in zip(filtered_documents, filtered_scores):
            print(f"Score: {1-score:.4f}, {doc}")
        print("\n")

    # Prepare the prompt for the model
    prompt = base_prompt.format(user_query=query, chunks_information="\n".join(documents))
    encoding = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.inference_mode():
        outputs = llm_model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
            num_beams=5,
            early_stopping=True,
        )

    generated_tokens = outputs[0][len(encoding.input_ids[0]):]
    generated_sequence = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    return generated_sequence

Try it out by changing the query! Try also to change `print_retrieved_documents` to `True` to see what documents are influencing the produced output.
>This operation takes ~15/30 seconds on our machine

In [ ]:
response = query_rag_model("What is Information Retrieval?", base_prompt, print_retrieved_documents=False)
print("Response: ", response)

# Evaluating model responses

The following section covers experiments divided into two different tasks:

1. How the model replies to **open-answer** questions
2. How the model replies to **multiple-choice** questions

For both test phase, questions were  stored into the json files described below.

The `quiz.json` file contains multiple-choice questions extracted from the *Multimedia and Information Retrieval* course's self-assessment quizzes. Below is an example of a question format:

```json
{ 
    "question": "What is a key limitation of the Rocchio algorithm when applying relevance feedback in Information Retrieval?",
    "answers": [
        "It only uses positive feedback from relevant documents",
        "It may suffer from query drift when non-relevant terms are included",
        "It assumes that all terms in non-relevant documents have negative weights",
        "It cannot handle queries with more than 10 terms"
    ],
    "correct": 2,
    "question_id": 1 
}, ...
```

Each field in the JSON structure has the following meaning:

<div align="center">

| **Field**       | **Content**                                     |
|------------------|-------------------------------------------------|
| `question`       | The question text                              |
| `answers`        | A list of four possible answers                |
| `correct`        | The index of the correct answer in `answers`   |
| `question_id`    | Unique identifier for the question         |

</div>

On the other hand, the `open_questions.json` file contains open-answer questions, where an entry looks like:

```json 
{
        "question_id": "1",
        "question": "What is Jaccard Coefficient?"
}, ...
```

where:

<div align="center">

| **Field**       | **Content**                                     |
|------------------|-------------------------------------------------|
| `question_id`    | Unique identifier of the question        |
| `question`       | The question text                              |

</div>


In [92]:
evaluation_folder_path = "./evaluation/"
quiz_file_path = evaluation_folder_path + "quiz.json"
questions_file_path = evaluation_folder_path + "open_questions.json"

We will use the already defined `extract_questions(file_path, question_type)` function to obtain both the necessary files

In [ ]:
quiz_questions = extract_questions(quiz_file_path, "mc")
print(f"Loaded {len(quiz_questions)} quiz questions.")
print(quiz_questions[0])

In [ ]:
open_questions = extract_questions(questions_file_path, "open")
print(f"Loaded {len(open_questions)} open questions.")
print(open_questions[0])

### Evaluating the model on open-questions

`process_open_questions(queries, folder_path)` is a function used to run experiments over a list of open-answer questions, taking as input the list of questions with `queries` and folder path `folder_path` for results storing.

1. Delete previous results if already done, create `res` folder otherwise
2. Process one-by-one queries via `response = query_rag_model(query, base_prompt)`

Note that:
* No results are returned since are already store on `.txt` files
* retrieve_relevant_documents is called only to write on .txt files documents retrieved, but it is also called into `query_rag_model` function

In [ ]:
def process_open_questions(queries, folder_path):
    """
    Processes a list of open questions, retrieves relevant documents, 
    and saves the responses to text files in a specified folder.
    
    Parameters:
    - queries (list): List of queries to process.
    - folder_path (str): Path to the folder where results will be saved.
    """
    results_folder = os.path.join(folder_path, "open-questions")
    if os.path.exists(results_folder):
        # Clear the folder if it exists
        for file_name in os.listdir(results_folder):
            file_path = os.path.join(results_folder, file_name)
            if os.path.isfile(file_path):
                os.remove(file_path)
    else:
        # Create the folder if it doesn't exist
        os.makedirs(results_folder)
        
    # Process each query and save the response
    for idx, query in enumerate(tqdm(queries, desc="Processing queries"), start=1):
        # Get the model's response
        response = query_rag_model(query['question'], base_prompt, print_retrieved_documents=False)

        # Construct the file path for the current query
        file_path = os.path.join(results_folder, f"open_question_{idx}.mmd")

        # Save the query, response, and documents to a text file
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(f"Query {idx}: {query['question']}\n\n")
            file.write(f"Reply {idx}: {response}")


# This array can be used as an example
# open_question_tmp = [
#     "What is Jaccard coefficient?",
#     "How does Document At A Time query processing work?",
#     "How does Term At A Time query processing work?"
# ]

process_open_questions(open_questions, evaluation_folder_path)

## TODO : Manca commento risultati domande aperte

### Evaluating the model on a closend-answer quiz

We also performed an experiments over a quiz file, to test:

* The model ability to reply with correct **information**
* Its ability to reply in a correct **format**

According to these objectives, the prompt for the LLM is changed since in this case we require a short and coincise reply.

After several tested prompts, the most performing one is reported below:

In [83]:
quiz_prompt =  '''You are an AI assistant for RAG.
Your task is to provide answers to user questions based solely on the provided contexts.
If and only if ALL of the provided context lacks sufficient information, respond with the digit "0".
Otherwise, provide only the index (1, 2, 3, or 4) of the correct answer based on the contexts.

Do not include any additional text, explanations, or speculations. Your response must consist of a single digit only.

Question: {user_query}

Contexts:
{chunks_information}
'''

The following function `run_quiz_with_rag(questions, evaluation_folder_path, quiz_prompt)` takes as input the questions, a folder to save results to, and the quiz prompt. It produces a run file for the model answer.

In [101]:
import time


def run_quiz_with_rag(questions, evaluation_folder_path, quiz_prompt):
    correct_count = 0
    error_count = 0
    results = {}

    # Generate a timestamped file for quiz runs
    current_time = time.strftime("%m%d-%H%M%S")
    quiz_run_folder = os.path.join(evaluation_folder_path, "RAG_quiz_runs")
    os.makedirs(quiz_run_folder, exist_ok=True)
    quiz_answers_file = os.path.join(quiz_run_folder, f"quiz_answers_{current_time}.txt")
    print(f"Saving quiz answers to: {quiz_answers_file}")

    # Progress bar for question processing
    pbar = tqdm(questions, total=len(questions), desc="Answering questions...", unit="question")
    for question in pbar:
        # Query the RAG model
        generated_sequence = query_rag_model(question['question'],quiz_prompt)
        # Extract the answer from the generated text
        match = re.search(r'\b[1-4]\b', generated_sequence)
        answer = match.group(0) if match else "0"  # Default to "0" if no valid answer is found

        # Write the results to a file
        with open(quiz_answers_file, "a", encoding="utf-8") as f:
            f.write(
                f"Question: {question['question']}\n"
                f"Answer: {answer}\n"
                f"Correct Answer: {question['correct']}\n"
                f"Generated Sequence: {generated_sequence}\n"
                f"--------------------------------------------------------------------\n\n"
            )

        # Update the results dictionary
        results[question['question_id']] = answer

        # Check answer validity
        if answer not in {"1", "2", "3", "4"}:
            error_count += 1
        else:
            # Check if the answer is correct
            if str(question['correct']) == answer:
                correct_count += 1
        
        # Update the progress bar
        pbar.set_postfix(Corrects=f"{correct_count}/{len(questions)}", Errors=error_count)

    print("-------------------------\tFINISHED RUN. Errors: ", error_count, "-------------------------")
    accuracy = (correct_count / len(questions)) * 100
    return results, accuracy


In [ ]:
create_quiz_run(quiz_questions)

# Gradio Interface <div align="center"><img src="https://www.gradio.app/_app/immutable/assets/gradio.CHB5adID.svg" alt="Gradio Logo" width="200"></div>

Lastly, we provide a **Gradio** interface to make interacting with the RAG pipeline both user-friendly and accessible. [Gradio](https://gradio.app) is a Python library that enables developers to quickly create customizable, interactive web-based interfaces. It is widely appreciated for its simplicity and flexibility.

The interface is launched in the next cell through the final command `demo.launch(debug=True)`

> This will also start a local server and generate a link (usually `http://127.0.0.1:7860`) that you can open in your browser. On Colab this will be different, but the you can find the url in the console output

The interface is straightforward:
- You can type your questions or prompts for the RAG pipeline into the **input field**.
- The interface estimated inference waiting time (based on the last response waiting time), then the generated answer.

If the interface is launched with the `debug=True` option (as in this case), it will also print the retrieved documents and their scores in the console; this can be useful for gaining insights into the system's behavior.

For mathematical formulas, `latex2mathml` is used: it is a Python library that convertes LateX formulas to MathML (an XML format for equations).

In [ ]:
import gradio as gr

latex_delimiters = [
    {"left": "\\[", "right": "\\]", "display": True},  # Formule in modalità display
    {"left": "\\(", "right": "\\)", "display": False},  # Formule inline
]

# Creazione dell'interfaccia Gradio con stile consistente
with gr.Blocks(css="""
    .input-box { border: 1px solid #ccc; border-radius: 4px; padding: 10px; margin: 10px 0; }
    .output-box { border: 1px solid #ccc; border-radius: 4px; padding: 10px; margin: 10px 0; background-color: #373535; height: auto; }

""") as demo:
    gr.Markdown("# RAG Model Query Interface")
    gr.Markdown("Ask questions to the RAG model and get answers based on the provided PDF context.")

    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(
                label="Enter your query",
                placeholder="Type your question here...",
                elem_classes=["input-box"]
            )

    with gr.Row():
        with gr.Column():
            output_text = gr.Markdown(
                label="Answer",
                elem_classes=["output-box"],
                latex_delimiters=latex_delimiters
                )
            

    input_text.submit(
        fn=query_rag_model,
        inputs=input_text,
        outputs=output_text
    )

# launch the interface
demo.launch(debug=True, show_error=True)